<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/Kondratev_Itog_Analitika_bolshih_dannyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Итоговая работа "Аналитика больших данных"

**Преподаватель:** Алексей Кузьмин

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

**1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы**



Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

**2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок**



Отберем фильмы, которые оценил данный пользователь

**3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь**



Для построения модели нам нужны признаки. В качестве таковых будем использовать:

* Год выхода

* Жанры
* Общее количество оценок

* Суммарную оценку




**4. Добавьте к датафрейму из задания 3 столбцы:**

  * По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

  * столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей



Теперь все готово и можно строить модель


**5. Сформируйте X_train, X_test, y_train, y_test**

**6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах**

**7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии**


Вторая часть практики Python связана со Spark’ом

**8. Загрузить данные в spark**

**9. Средствами спарка вывести среднюю оценку для каждого фильма**

**10. Посчитайте средствами спарка среднюю оценку для каждого жанра**

**11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)**


## Решение

**В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.**

### 1 - Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

**Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок**

In [52]:
import pandas as pd

# Ссылка на файл с оценками
ratings_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.data.csv"

# Загружаем файл с оценками в датафрейм
ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Ссылка на файл с фильмами
movies_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.item.csv"

# Загружаем файл с фильмами в датафрейм
movies = pd.read_csv(movies_url, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                            'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

**Смотрим файл по оценкам (ratings)**

In [53]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   movie_id   100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [54]:
# Список столбцов в датафрейме 'ratings', где есть пустые значения NaN
ratings.columns[ratings.isnull().any()].tolist()

[]

In [55]:
# Посмотрим на первые строки каждого датафрейма
ratings

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


```
u.data — полный набор данных u, 100 000 оценок 943 пользователей по 1682 элементам.
```
```
Каждый пользователь оценил не менее 20 фильмов. Пользователи и элементы
пронумерованы последовательно с 1. Данные случайным образом
заказал.
```

```
Столбцы:

user_id	                          item_id	      rating	      timestamp
идентификатор пользователя | идентификатор элемента | рейтинг | временная метка
```
```
Это список с разделителем табуляция sep='\t'
```
```
Временные метки представляют собой unix-секунды с 01.01.1970 UTC.
```

**Смотрим файл фильмам (movies)**

In [56]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   movie_title         1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb_URL            1679 non-null   object 
 5   unknown             1682 non-null   int64  
 6   Action              1682 non-null   int64  
 7   Adventure           1682 non-null   int64  
 8   Animation           1682 non-null   int64  
 9   Children            1682 non-null   int64  
 10  Comedy              1682 non-null   int64  
 11  Crime               1682 non-null   int64  
 12  Documentary         1682 non-null   int64  
 13  Drama               1682 non-null   int64  
 14  Fantasy             1682 non-null   int64  
 15  Film-Noir           1682 non-null   int64  
 16  Horror

In [57]:
# Список столбцов в датафрейме 'movies', где есть пустые значения NaN
movies.columns[movies.isnull().any()].tolist()

['release_date', 'video_release_date', 'IMDb_URL']

In [58]:
# print("Строки с пустым значением в столбце 'release_date':")
movies.loc[movies['release_date'].isnull()]

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# print("Строки с пустым значением в столбце 'video_release_date':")
movies.loc[movies['video_release_date'].isnull()]

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
# print("Строки с пустым значением в столбце 'IMDb_URL':")
movies.loc[movies['IMDb_URL'].isnull()]

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1357,1358,The Deadly Cure (1996),16-Sep-1996,NaN,NaN,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1358,1359,Boys in Venice (1996),24-Sep-1996,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# print("Строки с movie_title = 'unknown':")
movies.loc[movies['movie_title'] == 'unknown']

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# информация из таблицы ratings по фильму movie_title = 'unknown', movie_id = 267
ratings[ratings['movie_id'] == 267]

,user_id,movie_id,rating,timestamp
2172,130,267,5,875801239
3781,5,267,4,875635064
7245,268,267,3,875742077
12475,297,267,3,875409139
14756,319,267,4,875707690
15292,1,267,4,875692955
49295,532,267,3,875441348
93523,833,267,1,875655669
99723,422,267,4,875655986


In [63]:
movies

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


```
u.item - Информация фильмах:
```

```
Столбцы:

идентификатор фильма | название фильма | дата выпуска | дата выхода видео |
URL-адрес IMDb | неизвестный | Действие | Приключения | Анимация |
Детская | комедия | Преступность | Документальный | Драма | Фэнтези |
Фильм-Нуар | Ужасы | Музыкальный | Тайна | Романтика | Научная фантастика |
Триллер | Война | Западная |
```           
```
Это список с разделителем sep='|'
```
```
Последние 19 полей — жанры,

1 — фильм. относится к этому жанру

0 - означает, что это не так

фильмы могут быть внесколько жанров одновременно
```
```
Идентификаторы фильмов (movie_id) — это те, которые используются в наборе данных u.data.
```

### 2 - Средствами Pandas, используя dataframe `ratings`, найдите id пользователя, поставившего больше всего оценок

In [64]:
# Группируем данные по 'user_id' и считаем количество оценок каждого пользователя
# По умолчанию метод `value_counts()` в Pandas сортирует уникальные значения в порядке убывания количества их появлений.
user_ratings_count = ratings['user_id'].value_counts()

# Определяем top_user_id - ID пользователя с наибольшим количеством оценок
top_user_id = user_ratings_count.index[0]

print(f"ID пользователя с наибольшим количеством оценок: {top_user_id}")
print(f"Количество оценок пользователя: {user_ratings_count.values[0]}")


ID пользователя с наибольшим количеством оценок: 405
Количество оценок пользователя: 737


**Отберем фильмы, которые оценил данный пользователь**

### 3 - Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

Для построения модели нам нужны признаки. В качестве таковых будем использовать:
* Год выхода
* Жанры
* Общее количество оценок
* Суммарную оценку


In [65]:
# Вычисляем общее количество оценок и суммарную оценку от всех пользователей для каждого фильма
ratings['total_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('count')
ratings['sum_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('sum')

# Отображаем обновленный датафрейм ratings
ratings

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
0,196,242,3,881250949,117,467
1,186,302,3,891717742,297,1236
2,22,377,1,878887116,13,28
3,244,51,2,880606923,81,280
4,166,346,1,886397596,126,459
...,...,...,...,...,...,...
99995,880,476,3,880175444,160,483
99996,716,204,5,879795543,350,1342
99997,276,1090,1,874795795,37,89
99998,13,225,2,882399156,109,317


In [66]:
# Оставляем только фильмы, оцененные пользователем - 405 (top_user_id)
user_ratings = ratings[ratings['user_id'] == top_user_id]
user_ratings.head(3)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
12276,405,56,4,885544911,394,1600
12383,405,592,1,885548670,9,30
12430,405,1582,1,885548670,1,1


### 4 - Добавьте к датафрейму из задания 3 столбцы:

* Жанр
* Год выхода
* столбец с общим количеством оценок от всех пользователей на фильм - уже добавили на в 3-м задании
* столбец и суммарной оценкой от всех пользователей - уже добавили на в 3-м задании


In [67]:
# Добавляем к датафрейму столбцы 'жанр', 'год выхода'
user_ratings = user_ratings.merge(movies[['movie_id', 'release_date'] + list(movies.columns[5:24])], on='movie_id', how='left')
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,01-Jan-1994,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# Преобразование столбца с датой
user_ratings['release_date'] = pd.to_datetime(user_ratings['release_date'])
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,1994-01-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Извлечение года из даты и замена этим значением столбца release_date
user_ratings['release_date'] = user_ratings['release_date'].dt.year
user_ratings.head(1)

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,1994,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Теперь все готово и можно строить модель**

### 5 - Сформируйте X_train, X_test, y_train, y_test

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error

In [71]:
# Выделение признаков (X) и целевой переменной (y)
X = user_ratings[list(user_ratings.columns[4:26])]
y = user_ratings['rating']

In [72]:
X.head(1)

,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,394,1600,1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Разбиение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 6 - Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [74]:
# Создание модели линейной регрессии
model = LinearRegression()

# Обучение модели на данных
model.fit(X_train, y_train)

LinearRegression()

### 7 - Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [75]:
# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

In [76]:
# DataFrame для удобства сравнения
predictions = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred})
predictions.head()

,Фактическое значение,Предсказанное значение
669,1,1.136713
33,1,1.775713
549,1,1.083887
199,1,1.341517
264,1,3.029876


In [77]:
# Вычисление точности модели с использованием различных метрик
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE - корень из MSE

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R^2 Score: {r2:.2f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

Mean Absolute Error (MAE): 0.95
R^2 Score: 0.16
Mean Absolute Percentage Error (MAPE): 0.55
Root Mean Squared Error (RMSE): 1.30


Значения метрик ошибок (MAE, R^2, MAPE, RMSE) в контексте задачи предсказания рейтинга.

1. Mean Absolute Error (MAE) равный 0.95:
   - MAE измеряет среднюю абсолютную ошибку модели. Значение 0.95 означает, что в среднем модель ошибается на примерно 0.95 балла в оценке рейтинга. Чем ниже MAE, тем лучше.

2. R^2 Score равный 0.16:
   - R^2 Score или коэффициент детерминации измеряет объясненную вариацию моделью. Значение 0.16 говорит о том, что модель объясняет примерно 16% изменчивости целевой переменной. Чем ближе значение R^2 к 1, тем лучше модель объясняет данные.

3. Mean Absolute Percentage Error (MAPE) равный 0.55:
   - MAPE измеряет среднюю абсолютную процентную ошибку модели. Значение 0.55 означает, что модель в среднем ошибается на 55% от фактического значения рейтинга. Чем ниже MAPE, тем лучше.

4. Root Mean Squared Error (RMSE) равный 1.30:
   - RMSE измеряет среднеквадратичную ошибку модели. Значение 1.30 означает, что корень из среднеквадратичной ошибки составляет приблизительно 1.30 балла. Чем ниже RMSE, тем лучше модель предсказывает.

Исходя из значений метрик, модель, вероятно, не слишком точно предсказывает рейтинги и есть место для улучшения. Будет полезно анализировать причины ошибок и возможно рассмотреть использование других признаков или моделей для повышения точности предсказаний.

**Вторая часть практики Python связана со Spark’ом**




In [78]:
pip install pyspark


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 8 - Загрузить данные в spark

In [79]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

# Создаем сессию Spark
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()
sqlContext = SQLContext(spark)

/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [80]:
# Преобразуем Pandas DataFrame в Spark DataFrame
ratings_spark = sqlContext.createDataFrame(ratings)
movies_spark = sqlContext.createDataFrame(movies)

In [81]:
# Отображение первых строк Spark DataFrame
ratings_spark.show(3)

+-------+--------+------+---------+-----------------+---------------+
|user_id|movie_id|rating|timestamp|total_ratings_all|sum_ratings_all|
+-------+--------+------+---------+-----------------+---------------+
|    196|     242|     3|881250949|              117|            467|
|    186|     302|     3|891717742|              297|           1236|
|     22|     377|     1|878887116|               13|             28|
+-------+--------+------+---------+-----------------+---------------+
only showing top 3 rows



In [82]:
movies_spark.show(3)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|               NaN|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|               NaN|http

### 9 - Средствами спарка вывести среднюю оценку для каждого фильма

In [83]:
from pyspark.sql.functions import avg

# Рассчитываем среднюю оценку для каждого фильма
average_ratings = ratings_spark.groupBy("movie_id").agg(avg("rating").alias("average_rating"))
average_ratings.show(3)

24/03/04 08:50:14 WARN TaskSetManager: Stage 2 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+------------------+
|movie_id|    average_rating|
+--------+------------------+
|     474| 4.252577319587629|
|      29|2.6666666666666665|
|      26| 3.452054794520548|
+--------+------------------+
only showing top 3 rows



In [84]:
# Присоединяем информацию о фильмах к таблице средних оценок
result = average_ratings.join(movies_spark, "movie_id")
result.show(3)

24/03/04 08:50:15 WARN TaskSetManager: Stage 5 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|    average_rating|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      26| 3.452054794520548|Brothers McMullen...| 01-Jan-1995|               NaN|http://us.imdb.co...|      0|     0|        0|        0|       0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     

In [85]:
# Показываем результат
result.select("movie_id", "movie_title", "average_rating").show(3)

24/03/04 08:50:16 WARN TaskSetManager: Stage 11 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+--------------------+------------------+
|movie_id|         movie_title|    average_rating|
+--------+--------------------+------------------+
|      26|Brothers McMullen...| 3.452054794520548|
|      29|Batman Forever (1...|2.6666666666666665|
|     474|Dr. Strangelove o...| 4.252577319587629|
+--------+--------------------+------------------+
only showing top 3 rows



### 10 - Посчитайте средствами спарка среднюю оценку для каждого жанра

In [86]:
# Объединяем таблицы рейтингов и фильмов по movie_id
merged_data = ratings_spark.join(movies_spark, "movie_id")
merged_data.show(3)

24/03/04 08:50:17 WARN TaskSetManager: Stage 17 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+-------+------+---------+-----------------+---------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|user_id|rating|timestamp|total_ratings_all|sum_ratings_all|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-------+------+---------+-----------------+---------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      26|    686|     5|879546847|               73|            252|Brothers McMullen...| 01-Jan-1995|         

In [87]:
# Создаем новый DataFrame final_data на основе merged_data
final_data = merged_data

In [88]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, when

In [89]:
# Создаем список всех жанров
genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

genres

['unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [90]:
# Создаем новые столбцы для каждого жанра и вычисляем среднюю оценку по каждому жанру, только для фильмов с жанром, у которого значение равно 1
for genre in genres:
    filtered_data = final_data.filter(col(genre) == 1)
    avg_rating = filtered_data.groupBy().avg("rating").collect()[0][0]
    avg_rating_rounded = round(avg_rating, 2)  # Округляем до 2 знаков после запятой
    print(f"Средняя оценка по жанру {genre}: {avg_rating_rounded}")

24/03/04 08:50:19 WARN TaskSetManager: Stage 23 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


24/03/04 08:50:20 WARN TaskSetManager: Stage 32 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


Средняя оценка по жанру unknown: 3.2


Средняя оценка по жанру Action: 3.48


24/03/04 08:50:21 WARN TaskSetManager: Stage 41 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


...

Средняя оценка по жанру Sci-Fi: 3.56


24/03/04 08:50:35 WARN TaskSetManager: Stage 167 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.
24/03/04 08:50:36 WARN TaskSetManager: Stage 176 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


Средняя оценка по жанру Thriller: 3.51
Средняя оценка по жанру War: 3.82


24/03/04 08:50:37 WARN TaskSetManager: Stage 185 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


Средняя оценка по жанру Western: 3.61






11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)


In [91]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Группируем данные по movie_id и вычисляем средний рейтинг
avg_ratings = merged_data.groupBy("movie_id").agg(F.avg("rating").alias("avg_rating"))
avg_ratings.show(3)

24/03/04 08:50:38 WARN TaskSetManager: Stage 194 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+------------------+
|movie_id|        avg_rating|
+--------+------------------+
|      26| 3.452054794520548|
|      29|2.6666666666666665|
|     474| 4.252577319587629|
+--------+------------------+
only showing top 3 rows



In [92]:
# Присоединяем название фильма к датафрейму avg_ratings
avg_ratings_with_title = avg_ratings.join(movies_spark, "movie_id")
avg_ratings_with_title.show(3)

24/03/04 08:50:39 WARN TaskSetManager: Stage 200 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|        avg_rating|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+------------------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|      26| 3.452054794520548|Brothers McMullen...| 01-Jan-1995|               NaN|http://us.imdb.co...|      0|     0|        0|        0|       0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     

In [93]:
# Сортируем по убыванию среднего рейтинга и выводим топ-5 фильмов с самым высоким рейтингом
avg_ratings_with_title.select("movie_id", "movie_title", "avg_rating").sort("avg_rating", ascending=False).show(5)

24/03/04 08:50:41 WARN TaskSetManager: Stage 209 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+--------------------+----------+
|movie_id|         movie_title|avg_rating|
+--------+--------------------+----------+
|     814|Great Day in Harl...|       5.0|
|    1189|  Prefontaine (1997)|       5.0|
|    1599|Someone Else's Am...|       5.0|
|    1536|Aiqing wansui (1994)|       5.0|
|    1122|They Made Me a Cr...|       5.0|
+--------+--------------------+----------+
only showing top 5 rows



In [94]:
# Сортируем по возрастанию среднего рейтинга и выводим топ-5 фильмов с самыми низкими рейтингами
# (без указания направления сортировки она автоматически будет возрастающей).
avg_ratings_with_title.select("movie_id", "movie_title", "avg_rating").sort("avg_rating").show(5)

24/03/04 08:50:42 WARN TaskSetManager: Stage 218 contains a task of very large size (1004 KiB). The maximum recommended task size is 1000 KiB.


+--------+--------------------+----------+
|movie_id|         movie_title|avg_rating|
+--------+--------------------+----------+
|     784|Beyond Bedlam (1993)|       1.0|
|    1671|Further Gesture, ...|       1.0|
|     437|Amityville 1992: ...|       1.0|
|     599|Police Story 4: P...|       1.0|
|     314|3 Ninjas: High No...|       1.0|
+--------+--------------------+----------+
only showing top 5 rows



In [95]:
# Завершаем Spark сессию
spark.stop()